In [10]:
import slate
import urllib
import re

In [3]:
furl = urllib.urlopen('http://www.nature.com/ismej/journal/v10/n1/pdf/ismej2015100a.pdf')
with open( '/tmp/asdfasdfasdf.pdf', 'w' ) as ftempfile :
    ftempfile.write( furl.read() )
with open( '/tmp/asdfasdfasdf.pdf' ) as f :
    doc = slate.PDF(f)

### PDF is garbage

In this example, we are looking for a link to some source code :

[`http://prodege.jgi-psf.org//downloads/src`](http://prodege.jgi-psf.org//downloads/src)

However, in the PDF, the URL is line wrapped, so the `src` is lost.

In [20]:
urlre = re.compile( '(?P<url>https?://[^\s]+)' )

for page in doc :
    print urlre.findall( page )

[]
['http://prodege.jgi-psf.org//downloads/', 'http://prodege.jgi-psf.org,']
[]
['http://img.jgi.', 'http://www.nature.com/ismej)']


### PDF is garbage, continued

If we remove line breaks to fix URLs that have been wrapped, we discover
that the visible line breaks in the document do not correspond to actual
line breaks in the represented text. The result is random garbage. 

In [19]:
urlre = re.compile( '(?P<url>https?://[^\s]+)' )

for page in doc :
    print urlre.findall( page.replace('\n','') )

[]
['http://prodege.jgi-psf.org//downloads/availablerun', 'http://prodege.jgi-psf.org,which']
[]
['http://img.jgi.Cell', 'http://creativecommons.org/licenses/by/4.0/the', 'http://www.nature.com/ismej)The']


### Nope.

At this point, the author elects to flip a table. 

Let's try looking at the HTML version. I'll swipe some code from 
[Dive into Python](http://www.diveintopython.net/) here, because 
finding URLs in a HTML document is what is known as a "Solved Problem."

In [1]:
from sgmllib import SGMLParser

class URLLister(SGMLParser):
    def reset(self):                              
        SGMLParser.reset(self)
        self.urls = []

    def start_a(self, attrs):                     
        href = [v for k, v in attrs if k=='href']  
        if href:
            self.urls.extend(href)
            
def get_urls_from(url):
    url_list = []
    import urllib
    usock = urllib.urlopen(url)
    parser = URLLister()
    parser.feed(usock.read())         
    usock.close()      
    parser.close()                    
    map(url_list.append, 
        [item for item in parser.urls if item.startswith(('http', 'ftp', 'www'))])
    return url_list

Here are all the URLs in the document...

In [3]:
urls = get_urls_from('http://www.nature.com/ismej/journal/v10/n1/full/ismej2015100a.html')
urls

['http://www.isme-microbes.org/',
 'http://mts-isme.nature.com/cgi-bin/main.plex',
 'http://ad.doubleclick.net/N285/jump/ismej.nature.com/;abr=!NN2;type=sc;artid=ismej2015100a;issue=1;pos=top;subjmeta=631;subjmeta=208;subjmeta=212;techmeta=45;techmeta=23;sz=728x90;tile=1;ord=123456789?',
 'http://prodege.jgi-psf.org//downloads/src',
 'http://prodege.jgi-psf.org',
 'http://dx.doi.org/10.1089/cmb.2012.0021',
 'http://www.ncbi.nlm.nih.gov/entrez/query.fcgi?holding=npg&amp;cmd=Retrieve&amp;db=PubMed&amp;list_uids=22506599&amp;dopt=Abstract',
 'http://links.isiglobalnet2.com/gateway/Gateway.cgi?&amp;GWVersion=2&amp;SrcAuth=Nature&amp;SrcApp=Nature&amp;DestLinkType=FullRecord&amp;KeyUT=000303811800001&amp;DestApp=WOS_CPL',
 'http://chemport.cas.org/cgi-bin/sdcgi?APP=ftslink&action=reflink&origin=npg&version=1.0&coi=1:CAS:528:DC%2BC38XmsFOmt7k%3D&pissn=1751-7362&pyear=2016&md5=186ecb2f73d6216debcdc0dc83436dfe',
 'http://dx.doi.org/10.1186/1471-2105-10-421',
 'http://www.ncbi.nlm.nih.gov/entre

Bleh. That is mostly links in the references, ads and navigation cruft 
from the journal's content mismanagement system. Because their system
is heinously *ad hoc*, there is no base URL. So, we're forced to use an 
*ad hoc* exclusion list.

In [4]:
excluded = [ 'http://www.nature.com',
             'http://dx.doi.org',
             'http://www.ncbi.nlm.nih.gov',
             'http://creativecommons.org',
             'https://s100.copyright.com',
             'http://mts-isme.nature.com',
             'http://www.isme-microbes.org',
             'http://ad.doubleclick.net',
             'http://mse.force.com',
             'http://links.isiglobalnet2.com',
             'http://www.readcube.com',
             'http://chemport.cas.org',
             'http://publicationethics.org/',
             'http://www.natureasia.com/'
           ]

def novel_url( url ) :
    for excluded_url in excluded :
        if url.startswith( excluded_url ) :
            return False
    return True

filter( novel_url, urls )

['http://prodege.jgi-psf.org//downloads/src',
 'http://prodege.jgi-psf.org',
 'http://img.jgi.doe.gov/w/doc/SingleCellDataDecontamination.pdf']

Much better. Now, let's see if these exist...

In [5]:
import requests

for url in filter( novel_url, urls ) :
    request = requests.get( url )
    if request.status_code == 200:
        print 'Good : ', url
    else:
        print 'Fail : ', url

Good :  http://prodege.jgi-psf.org//downloads/src
Good :  http://prodege.jgi-psf.org
Fail :  http://img.jgi.doe.gov/w/doc/SingleCellDataDecontamination.pdf


Looks like this will work, though we'll need to make a hand-curated list of
excluded URLs. Othersise, the counts of dead links could be badly skewed by
any issues within the journal's content mismanagement system, ad servers and
other irrelevent crud.

### Walking through Zotero

Let's try walking through the publications in a Zotero library...

In [8]:
from pyzotero import zotero
api_key    = open( 'zotero_api_key.txt' ).read().strip()
library_id = open( 'zotero_api_userID.txt' ).read().strip()
library_type = 'group'
group_id = '405341' # microBE.net group ID

zot = zotero.Zotero(group_id, library_type, api_key)
items = zot.top(limit=5)
# we've retrieved the latest five top-level items in our library
# we can print each item's item type and ID
for item in items:
    #print('Item: %s | Key: %s') % (item['data']['itemType'], item['data']['key'])
    print item['data']['key'], ':', item['data']['title']

QC9BAHIK : ProDeGe: a computational protocol for fully automated decontamination of genomes
E7S5UR96 : In search of non-photosynthetic Cyanobacteria
T9GDRBT5 : Evidence-based recommendations on storing and handling specimens for analyses of insect microbiota
BJJUJW48 : Cautionary tale of using 16S rRNA gene sequence similarity values in identification of human-associated bacterial species
QD3JS59Z : ConStrains identifies microbial strains in metagenomic datasets


So far so good. Let's have a look at the `url` attribute...

In [47]:
for item in items:
    print item['data']['key'], ':', item['data']['url']

QC9BAHIK : http://www.nature.com/ismej/journal/v10/n1/full/ismej2015100a.html
E7S5UR96 : http://espace.library.uq.edu.au/view/UQ:368958
T9GDRBT5 : https://peerj.com/articles/1190
BJJUJW48 : 
QD3JS59Z : http://www.nature.com/nbt/journal/v33/n10/full/nbt.3319.html


Well, it looks like not all resources have URLs. Let's try looping over
some of these and extracting links...

In [9]:
for item in items:
    paper_url = item['data']['url']
    if paper_url.startswith( 'http' ) :
        link_urls = get_urls_from( paper_url )
        print item['data']['key']
        for url in filter( novel_url, link_urls ) :
            print '    ', url

QC9BAHIK
     http://prodege.jgi-psf.org//downloads/src
     http://prodege.jgi-psf.org
     http://img.jgi.doe.gov/w/doc/SingleCellDataDecontamination.pdf
E7S5UR96
     http://www.uq.edu.au/
     http://www.uq.edu.au/
     http://www.uq.edu.au/contacts/
     http://www.uq.edu.au/study/
     http://www.uq.edu.au/maps/
     http://www.uq.edu.au/news/
     http://www.uq.edu.au/events/
     http://www.library.uq.edu.au/
     http://my.uq.edu.au/
     http://ezproxy.library.uq.edu.au/login?url=http://dx.doi.org/10.14264/uql.2015.855
     http://espace.library.uq.edu.au/list/author/Soo%2C+Rochelle+Melissa/
     http://espace.library.uq.edu.au/list/?cat=quick_filter&search_keys%5Bcore_70%5D=School of Chemistry and Molecular Biosciences
     http://espace.library.uq.edu.au/list/subject/452051/
     http://espace.library.uq.edu.au/list/subject/452105/
     http://espace.library.uq.edu.au/list/?cat=quick_filter&search_keys%5B0%5D=Melainabacteria
     http://espace.library.uq.edu.au/list/?cat=qu

IOError: [Errno socket error] timed out

Clearly, we need to expand the excluded URL list. And we need to match 
domains, not URLs.

In [22]:
excluded = [ 'nature.com',
             'doi.org',
             'ncbi.nlm.nih.gov',
             'creativecommons.org',
             'copyright.com',
             'isme-microbes.org',
             'doubleclick.net',
             'force.com',
             'isiglobalnet2.com',
             'readcube.com',
             'cas.org',
             'publicationethics.org',
             'natureasia.com',
             'uq.edu.au',
             'edx.org',
             'facebook.com',
             'instagram.com',
             'youtube.com',
             'flickr.com',
             'twitter.com',
             'go8.edu.au',
             'google.com',
             'vimeo.com',
             'peerj.com',
             'mendeley.com',
             'cloudfront.net',
             'webofknowledge.com',
             'sciencedirect.com',
             'aol.com',
             'pinterest.com',
             'scopus.com',
             'live.com',
             'exlibrisgroup.com',
             'usyd.edu.au',
             'academicanalytics.com',
             'microbiomedigest.com',
             'ask.com',
             'sogou.com',
             'ou.com',
             'du.edu',
             'ru.nl',
             'freshdesk.com',
             'caltech.edu',
             'traackr.com',
             'adobe.com',
             'linkedin.com',
             'feedly.com',
             'google.co.uk',
             'glgoo.org',
             'library.wisc.edu',
             'lib.fsu.edu',
             'library.illinois.edu',
             'exchange.ou.edu',
             'lib.noaa.gov',
             'innocentive.com',
             'sfx.kcl.ac.uk',
             'sfx.unimi.it',
             'lib.utexas.edu',
             'orcid.org',
           ]

def novel_url( url ) :
    for excluded_url in excluded :
        if url.__contains__( excluded_url ) :
            return False
    return True

This excluded list is getting sloppy as the author slowly lapses into 
a vegitative state, but we'll push on anyway.

In [23]:
for item in items:
    paper_url = item['data']['url']
    if paper_url.startswith( 'http' ) :
        try :
            link_urls = get_urls_from( paper_url )
            print item['data']['key']
            for url in list(set(filter( novel_url, link_urls ))) :
                print '    ', url
        except IOError :
            print item['data']['key'], 'FAILED'

QC9BAHIK
     http://img.jgi.doe.gov/w/doc/SingleCellDataDecontamination.pdf
     http://prodege.jgi-psf.org
     http://prodege.jgi-psf.org//downloads/src
E7S5UR96 FAILED
T9GDRBT5
     http://had.co.nz/ggplot2/book
     http://CRAN.R-project.org/package=vegan
     http://www.tobinhammer.com/publications-and-twitter-feed.html
QD3JS59Z
     https://bitbucket.org/luo-chengwei/constrains
     http://hmpdacc.org/resources/tools_protocols.php


Some journals aggressivly ban and throttle IPs, so this process gets slow
and awful, but it works. Let's check these for dead links...

In [25]:
for item in items:
    paper_url = item['data']['url']
    if paper_url.startswith( 'http' ) :
        try :
            link_urls = get_urls_from( paper_url )
            print item['data']['key']
            for url in list(set(filter( novel_url, link_urls ))) :
                request = requests.get( url )
                if request.status_code == 200:
                    print '   Good : ', url
                else:
                    print '   Fail : ', url
        except IOError :
            print item['data']['key'], 'FAILED'

QC9BAHIK
   Fail :  http://img.jgi.doe.gov/w/doc/SingleCellDataDecontamination.pdf
   Good :  http://prodege.jgi-psf.org
   Good :  http://prodege.jgi-psf.org//downloads/src
E7S5UR96 FAILED
T9GDRBT5
   Fail :  http://had.co.nz/ggplot2/book
   Good :  http://CRAN.R-project.org/package=vegan
   Good :  http://www.tobinhammer.com/publications-and-twitter-feed.html
QD3JS59Z
   Good :  https://bitbucket.org/luo-chengwei/constrains
   Good :  http://hmpdacc.org/resources/tools_protocols.php


I guess that'll do for a proof of concept.